In [25]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pymongo import MongoClient
import concurrent.futures
import srt

client = MongoClient(uri)

class Model:
    def __init__(self, system_prompt):
        self.system = system_prompt
        self.prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system),
                MessagesPlaceholder("examples", optional=True),
                ("human", "follow the system prompt to deal with belowing content: {question}"),
            ]
        )
        self.llm = ChatOpenAI(model="gpt-4o", temperature=0)
        self.llm_chain = self.prompt | self.llm

    def ask_question(self, question):
        ai_message = self.llm_chain.invoke(question)
        return ai_message.content
    
    def parallel_questions(self, questions):
        with concurrent.futures.ThreadPoolExecutor() as executor:
            answers = list(executor.map(self.ask_question, questions))
        return answers

def get_subtitles(content):
    content = content.replace("，", " ") # goodder 的 srt 格式與常見格式不同，需要先處理
    subtitles = list(srt.parse(content))
    return subtitles

def split_subtitles(subtitles, max_length=2700):
    timestamp_content = []
    transcript = []
    timestamp_content_slice = ""
    transcript_slice = ""

    for subtitle in subtitles:
        temp_sub = f"{subtitle.index}\n{subtitle.start} --> {subtitle.end}\n{subtitle.content}\n\n"
        timestamp_content_slice += temp_sub
        transcript_slice += subtitle.content + " "
        
        if len(timestamp_content_slice) > max_length:
            timestamp_content.append(timestamp_content_slice)
            transcript.append(transcript_slice)
            timestamp_content_slice = ""
            transcript_slice = ""

    # Append the remaining slices to the lists
    if timestamp_content_slice:
        timestamp_content.append(timestamp_content_slice)
    if transcript_slice:
        transcript.append(transcript_slice)
    return timestamp_content, transcript

/Users/michael/miniconda3/envs/goodder_env/lib/python3.9/site-packages/pymongo/uri_parser.py:313: UserWarning: authmechanism must be in ('GSSAPI', 'MONGODB-X509', 'MONGODB-OIDC', 'SCRAM-SHA-1', 'DEFAULT', 'SCRAM-SHA-256', 'MONGODB-CR', 'PLAIN', 'MONGODB-AWS')
  return get_validated_options(opts, warn)


In [26]:
# Get the example document
db = client["goodder_event"]
collection = db["ai-att-table"]
query = {"full_duration": {"$gt": 4000}}
results = collection.find(query).limit(1)
content = results[0]["transcript"]["text"]

# Get the subtitles
subtitles = get_subtitles(content)
timestamp_content, transcript = split_subtitles(subtitles)
print(timestamp_content[0], transcript[0])

1
0:00:00 --> 0:00:05
跟你旁边的家人说我们都要长大成熟

2
0:00:05 --> 0:00:10
我们一起祷告

3
0:00:10 --> 0:00:12
亲爱的主耶稣

4
0:00:12 --> 0:00:14
帮助我

5
0:00:14 --> 0:00:17
越来越爱你

6
0:00:17 --> 0:00:21
越来越可慕你

7
0:00:21 --> 0:00:24
奉耶稣的名祷告

8
0:00:24 --> 0:00:27
一起说

9
0:00:27 --> 0:00:29
阿弥陀佛

10
0:00:29 --> 0:00:32
我们拍手欢迎天父耶稣圣灵

11
0:00:32 --> 0:00:34
来当我们的灯神

12
0:00:34 --> 0:00:36
耶

13
0:00:36 --> 0:00:38
祝我们欢迎你

14
0:00:38 --> 0:00:40
祝我们欢迎你

15
0:00:40 --> 0:00:42
阿弥陀佛

16
0:00:42 --> 0:00:46
很开心我们今天可以先用喜乐来敬拜我们的神

17
0:00:46 --> 0:00:49
让我们的声音直达天上

18
0:00:49 --> 0:00:51
讨天父爸爸的喜悦好不好

19
0:00:51 --> 0:00:53
好

20
0:00:53 --> 0:00:55
我们来拍手

21
0:00:57 --> 0:01:02
阿弥陀佛

22
0:01:02 --> 0:01:05
我的盼望

23
0:01:05 --> 0:01:08
就在耶稣基督里

24
0:01:08 --> 0:01:10
在你里面

25
0:01:10 --> 0:01:13
满有彼岸

26
0:01:13 --> 0:01:15
阿利路亚

27
0:01:15 --> 0:01:18
神明确认

28
0:01:18 --> 0:01:21
就在耶稣基督里

29
0:01:21 --> 0:01:26
永驻在我里面活着

30
0:01:26 --> 0:01:29
我宣告在我里面

31
0:01:29 --> 0:01:33
祢比世上一切更有能力

32
0:01:33 --> 0:01:38
我不输在世界我靠近

In [51]:
# task 1: Summarize the transcript
summarizing_system_prompt = """
As a professional YouTube Shorts editor, select about one minute of continuous and narrative-rich content from a longer video, adhering to these requirements:

Objective: Choose sentences that are temporally continuous and narrative-rich.
Restrictions: Do not include any AI-generated text.
Format: Report the exact locations of these sentences (e.g., 0:05:54 --> 0:05:56
Hallelujah

0:05:56 --> 0:05:57
祝大家的荣耀归于你

0:05:57 --> 0:05:59
谢谢主).

The response do not include any introductory or explanatory text.
"""

only_timestamp_version = """
As a professional YouTube Shorts editor, select about one minute of continuous and narrative-rich content from a longer video, adhering to these requirements:

Objective: Choose sentences that are temporally continuous and narrative-rich.
Restrictions: Do not include any AI-generated text.
Format: Report the exact locations of these sentences using timestamps (e.g., from 00:05:30 to 00:05:45).

The response do not include any introductory or explanatory text.
"""

summary_model = Model(summarizing_system_prompt)
# summary_list = summary_model.parallel_questions(transcript)
# print(summary_list)

result = summary_model.ask_question(timestamp_content[-1])

In [53]:
print(result)

1:56:46 --> 1:56:48
主耶稣求祢赐下可牧

1:56:48 --> 1:56:52
在我们每一位弟兄姐妹的生命里面

1:56:52 --> 1:56:54
Amen

1:56:54 --> 1:56:57
让我们可牧耶稣

1:56:57 --> 1:56:59
可牧耶稣

1:56:59 --> 1:57:02
因为耶稣是生命的粮

1:57:02 --> 1:57:04
Amen

1:57:04 --> 1:57:10
让我们花时间在这永生的食物身上

1:57:10 --> 1:57:13
主啊求祢救我们

1:57:13 --> 1:57:22
救我们脱离这些属地的属情欲的属鬼魔的这些垃圾食物

1:57:22 --> 1:57:25
主啊救我们救我们

1:57:25 --> 1:57:29
让我们全教会一起来可牧耶稣

1:57:29 --> 1:57:30
Amen

1:57:30 --> 1:57:32
让我们的灵人长大

1:57:32 --> 1:57:34
让我们做一个对的人

1:57:34 --> 1:57:35
Amen

1:57:35 --> 1:57:38
让我们的侍奉是蒙越纳的侍奉

1:57:38 --> 1:57:39
Amen

1:57:39 --> 1:57:42
谢谢耶稣我们将祷告奉耶稣基督的名

1:57:42 --> 1:57:44
Amen


In [59]:
# task 2: Add punctuation marks to transcript
punctuation_system_prompt = """
Add punctuation marks to the following text. If a sentence is already punctuated, leave it as is.
"""
punctuation_model = Model(punctuation_system_prompt)
punctuated_list = punctuation_model.parallel_questions(transcript[:2])
# punctuated_list = punctuation_model.ask_question(transcript[0])
punctuated_content = "".join(punctuated_list)

In [61]:
punctuated_content

'跟你旁边的家人说，我们都要长大成熟。我们一起祷告：亲爱的主耶稣，帮助我，越来越爱你，越来越可慕你。奉耶稣的名祷告，一起说，阿弥陀佛。我们拍手欢迎天父、耶稣、圣灵，来当我们的灯神。耶，祝我们欢迎你，祝我们欢迎你，阿弥陀佛。\n\n很开心我们今天可以先用喜乐来敬拜我们的神，让我们的声音直达天上，讨天父爸爸的喜悦，好不好？好，我们来拍手，阿弥陀佛。\n\n我的盼望，就在耶稣基督里。在你里面，满有彼岸。阿利路亚，神明确认，就在耶稣基督里，永驻在我里面活着。我宣告在我里面，祢比世上一切更有能力。我不输在世界，我靠近耶稣基督。我宣告在我里面，有活水涌流，满有能力。有耶稣在我里面活着，我勇敢向祢。祝在祢面前，永驻在我里面。没有惧怕能战胜仇敌，永驻在我里面。没有惧怕，爱得善忧郁。永驻在我里面，没有惧怕能战胜仇敌。永驻在我里面，没有惧怕，爱得善忧郁。\n\n来，跟你的宝贝一个掌开拜托。我们的神是能力之源。我们的神是能力之源。大家来跟拜托，来，我们用您里面来祷告，然后宣告。不论在什么样困难的环境，来到我们这个正好的环境当中，不要为了要去消逝我们的力量。不要为了要去消逝我们的力量。我们请看护我们，我们愿传承主。祝祢要用我们的力量，祈祷我们。祝祢要用我们的力量，祝祢要用我们的力量，继续来敬拜。\n\n我的盼望，就在耶稣基督里。在祢里面，满有平安。生命前沿，就在耶稣基督里，永驻在我里面。火车再宣告一次：我的盼望，我的盼望，就在耶稣基督里。在祢里面，满有平安。生命前沿，就在耶稣基督里，永驻在我里面。火车再宣告：我宣告，在我里面，祢是上一线更勇的力量。我付出这世界，我靠近耶稣基督。我宣告。Follow the system prompt to deal with the following content:\n\n在我里面，火车拥有满有能力。有耶稣在我里面，活着我勇敢在里。再一次，我宣告，在我里面，祢是上一线更勇的力量。我付出这世界，我靠近耶稣基督。我宣告，在我里面，火车拥有满有能力。有耶稣在我里面，活着我勇敢在里。高举你的手，永驻在我里面。没有奇葩能战胜中医，永驻在我里面。没有奇葩能战胜中医，永驻在我里面。没有奇葩能战胜中医，宣告。永驻在我里面，没有奇葩能战胜中医。永驻在我里面，没有奇葩能战胜中医。永驻在我里面，没有奇葩能战胜中医，宣告。我宣告在我里面，你，你写上一些感动的语言。我不输这时间